In [ ]:
#Include all packages
import os
import cv2
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D
from tensorflow.keras.layers import add, concatenate, BatchNormalization, LeakyReLU, ZeroPadding2D
from tensorflow.keras.losses import BinaryCrossentropy, MeanSquaredError
from tensorflow.keras.metrics import MeanIoU
import pandas as pd


In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
#  import zipfile

# with zipfile.ZipFile('/content/drive/MyDrive/DL/FinalProject/signDatabasePublicFramesOnly.zip', 'r') as zip_ref:
#     zip_ref.extractall('./DataSet')


In [ ]:
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
physical_devices = tf.config.list_physical_devices('GPU')
print(physical_devices)
if len(physical_devices) > 0:  # If you have at least one "configured" GPU, let's use it; otherwise, pass
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
    print("Using GPU")


In [ ]:
def LoadDataSet(dataSetFolderPath: str):
    images = []
    newAnnotations = []
    annotationsFilePath = dataSetFolderPath+"/allAnnotations.csv"
    annotationsDataFrame = pd.read_csv(annotationsFilePath,sep=";")
    width, height = 416, 416
    for index, row in annotationsDataFrame[1:].iterrows():
        image = cv2.imread(dataSetFolderPath+"/"+row[0])
        # cv2.imshow("image", image)
        resizedImage  = cv2.resize(image, (width, height), interpolation=cv2.INTER_AREA)
        # cv2.imshow("resized_img", resized_img)
        images.append(resizedImage)
        x1,y1,x2,y2 = row[2],row[3],row[4],row[5]
        x1, y1 = x1 / image.shape[1], y1 / image.shape[0]
        x2, y2 = x2 / image.shape[1], y2 / image.shape[0]
        newAnnotations.append([row[1], x1,y1,x2,y2])
    uniqueSigns =  annotationsDataFrame['Annotation tag'].unique()
    del annotationsDataFrame
    X_train, X_val, y_train, y_val = train_test_split(images, newAnnotations, test_size=0.3, random_state=42)

    # Create TensorFlow Datasets
    trainDataSet = tf.data.Dataset.from_tensor_slices((np.array(X_train), np.array(y_train)))
    valDataSet = tf.data.Dataset.from_tensor_slices((np.array(X_val), np.array(y_val)))
    return trainDataSet, valDataSet, uniqueSigns



In [ ]:
def _conv_block(x, num_filters, kernel_size, strides=1):
    x = Conv2D(num_filters, kernel_size, strides=strides, padding='same')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.1)(x)
    return x


In [ ]:
def _residual_block(x, num_filters):
    shortcut = x
    x = _conv_block(x, num_filters // 2, 1)
    x = _conv_block(x, num_filters, 3)
    x = add([shortcut, x])
    return x

In [ ]:
def _create_darknet53(input_shape):
    inputs = Input(shape=input_shape)

    x = _conv_block(inputs, 32, 3)
    x = _conv_block(x, 64, 3, strides=2)

    for _ in range(1):
        x = _residual_block(x, 64)

    x = _conv_block(x, 128, 3, strides=2)

    for _ in range(2):
        x = _residual_block(x, 128)

    x = _conv_block(x, 256, 3, strides=2)

    for _ in range(8):
        x = _residual_block(x, 256)

    route1 = x
    x = _conv_block(x, 512, 3, strides=2)

    for _ in range(8):
        x = _residual_block(x, 512)

    route2 = x
    x = _conv_block(x, 1024, 3, strides=2)

    for _ in range(4):
        x = _residual_block(x, 1024)

    return Model(inputs, (route1, route2, x), name='darknet53')

In [ ]:

def _create_yolov3(input_shape, num_classes):
    darknet = _create_darknet53(input_shape[:-1] + (3,))

    x = darknet.output[2]
    x = _conv_block(x, 512, 1)
    x = _conv_block(x, 1024, 3)
    x = _conv_block(x, 512, 1)
    x = _conv_block(x, 1024, 3)
    x = _conv_block(x, 512, 1)

    y1 = _conv_block(x, 1024, 3)
    y1 = Conv2D(3 * (num_classes + 5), 1, padding='same')(y1)
    print("y1 shape:", y1.shape)

    x = _conv_block(x, 256, 1)
    x = UpSampling2D(2)(x)
    x = concatenate([x, darknet.output[1]])

    x = _conv_block(x, 256, 1)
    x = _conv_block(x, 512, 3)
    x = _conv_block(x, 256, 1)
    x = _conv_block(x, 512, 3)
    x = _conv_block(x, 256, 1)

    y2 = _conv_block(x, 512, 3)
    y2 = Conv2D(3 * (num_classes + 5), 1, padding='same')(y2)
    print("y2 shape:", y2.shape)

    x = _conv_block(x, 128, 1)
    x = UpSampling2D(2)(x)
    x = concatenate([x, darknet.output[0]])

    x = _conv_block(x, 128, 1)
    x = _conv_block(x, 256, 3)
    x = _conv_block(x, 128, 1)
    x = _conv_block(x, 256, 3)
    x = _conv_block(x, 128, 1)

    y3 = _conv_block(x, 256, 3)
    y3 = Conv2D(3 * (num_classes + 5), 1, padding='same')(y3)
    print("y3 shape:", y3.shape)

    return Model(darknet.input, (y1, y2, y3), name='yolov3')


In [ ]:
def setup_yolov3_loss_and_metric(yolov3_model, num_classes, num_anchors):
    # num_classes = 3  # Update this value according to the number of traffic sign classes in your dataset
    # num_anchors = 3  # The number of anchor boxes used in YOLOv3 (e.g., 3 for tiny YOLOv3)

    # Define the loss components
    objectness_loss = BinaryCrossentropy()
    class_loss = BinaryCrossentropy()
    bbox_loss = MeanSquaredError()

    # Define the metric for evaluation (mean Intersection over Union)
    mean_iou = MeanIoU(num_classes=num_classes)

    def yolov3_loss(y_true, y_pred):
        # Extract the ground truth and predictions for objectness, class, and bbox
        y_true_objectness, y_true_class, y_true_bbox = tf.split(y_true, [num_anchors, num_classes, 4], axis=-1)
        y_pred_objectness, y_pred_class, y_pred_bbox = tf.split(y_pred, [num_anchors, num_classes, 4], axis=-1)

        # Calculate the individual losses
        objectness_loss_value = objectness_loss(y_true_objectness, y_pred_objectness)
        class_loss_value = class_loss(y_true_class, y_pred_class)
        bbox_loss_value = bbox_loss(y_true_bbox, y_pred_bbox)

        # Combine the losses
        total_loss = objectness_loss_value + class_loss_value + bbox_loss_value
        return total_loss

    return yolov3_loss, mean_iou


In [ ]:

def train_yolov3_model(yolov3_model, trainDataSet, valDataSet, loss_function, metric):
    # Set up the loss and metric

    # Compile the model with the loss function and metric
    yolov3_model.compile(optimizer='adam', loss=loss_function, metrics=[metric])

    # Train the model on the dataset
    history = yolov3_model.fit(
        trainDataSet.batch(32),
        epochs=100,
        validation_data=valDataSet.batch(32),
        callbacks=[
            tf.keras.callbacks.ModelCheckpoint('yolov3_best_weights.h5', save_best_only=True, monitor='val_loss'),
            tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, verbose=1, min_lr=1e-6),
            tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, verbose=1)
        ]
    )

    return history


In [ ]:
trainDataSet, valDataSet, uniqueSigns = LoadDataSet("./DataSet")


In [ ]:

input_shape = (416, 416, 3)
num_classes = len(uniqueSigns)  
print(num_classes)
# Create the YOLOv3 model
yolov3_model = _create_yolov3(input_shape, num_classes)

In [ ]:
# yolov3_model.compile(optimizer='adam', loss=yolov3_loss, metrics=[mean_iou])


In [ ]:

for i in range(1,9):
    try:
        print(i)
        num_anchors = i
        yolov3_loss, mean_iou = setup_yolov3_loss_and_metric(yolov3_model, num_classes, num_anchors)

        trained_model = train_yolov3_model(yolov3_model, trainDataSet, valDataSet, yolov3_loss, mean_iou)
    except:
        print("error")